In [25]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./")

import torch
from torch import nn
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
class TestNet(nn.Module):
    '''The network from https://arxiv.org/pdf/1606.08921v3.pdf'''
    def __init__(
            self, 
            in_channels,  # Number of Channels in our Focal Stack
            out_channels,  # Number of Channels in the output image, which is 1, grayscale
            kernel_size,  # Figure 10 in the paper shows filter sizes. read what is in the paper [Experiments 5.1.]
            num_filters,  # the number of filters / feature maps per layer. [Experiments 5.1.] More is better and more expensive.
            step_size,  # the step size of the residual connections. Fig. 12. The PSNR values on the validation set during training.
        ):
        super(TestNet, self).__init__()
        self.conv = nn.Conv2d(
            in_channels, 
            num_filters, 
            kernel_size, 
        )

        self.deconv = nn.ConvTranspose2d(
            num_filters, 
            out_channels, 
            kernel_size, 
        )

    def forward(self, x):
        # WARNIGN! hacky adaptation. The residual must have same shape to what it is added to.
        # taking the mean works, but I dont know if that breaks the whole thing. Could work though, worth a try.
        residual = x.mean(axis=1)

        out = self.conv(x)
        out = self.deconv(out)

        out += residual
        out = F.relu(out)
        return out


batch_size = 1
C = num_focal_planes = 5
H, W = 512,512

model = TestNet(
    in_channels=C,
    out_channels=1,
    kernel_size=3,
    num_filters=10,
    step_size=None
)

x = torch.ones(torch.Size([batch_size,C,H,W]))
y = model(x)

assert y.shape == torch.Size([batch_size, 1, H, W])
